In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Kafka-Spark Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

In [4]:
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

print("Streaming is running...")
query.awaitTermination()

24/12/18 13:36:50 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a3c1f113-439e-4e46-bb15-e6fd15537a4e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Streaming is running...
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------------+
|        value|
+-------------+
|test messeage|
+-------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+------------+
|       value|
+------------+
|test message|
+------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|test massage|
+--------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+---------------+
|          value|
+---------------+
|cardiac massage|
+---------------+



KeyboardInterrupt: 

# 옵션 설정 테스트

In [ ]:
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='3 seconds') \
    .start()

print("Streaming is running...")
query.awaitTermination()

# 데이터 처리 테스트

In [ ]:
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

# 모든 실행 중인 쿼리를 확인하고 중지
for query in spark.streams.active:
    if query.name == "kafka_messages":
        query.stop()
        
query = kafka_data.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("kafka_messages") \
    .start()

print("Streaming is running...")

import time
while True:
    spark.sql("SELECT * FROM kafka_messages").show()
    time.sleep(5)